<a href="https://colab.research.google.com/github/chetools/CHE4061_Fall2024/blob/main/NRTL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget -N -q https://raw.githubusercontent.com/chetools/chetools/main/tools/che5.ipynb -O che5.ipynb
!pip install importnb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 1.8 MB/s eta 0:00:00


In [53]:
from importnb import Notebook
with Notebook():
    from che5 import Props

import numpy as np
import jax
import jax.numpy as jnp
jax.config.update("jax_enable_x64", True)
from scipy.optimize import root_scalar

In [41]:
p=Props(['Acetone','Ethanol', 'Isopropanol', 'Water'])

In [47]:
def gamma(x,T):
    tau = p.NRTL_A + p.NRTL_B/T + p.NRTL_C*np.log(T) + p.NRTL_D*T
    G=np.exp(-p.NRTL_alpha*tau)
    xG = x@G
    xtauG_xG = (x@(tau*G))/xG
    return np.exp(xtauG_xG + x@((G*(tau - xtauG_xG[None,:])/xG[None,:]).T))

In [45]:
x=np.array([0.1, 0.2, 0.3, 0.4])
T=350.

In [50]:
gamma(x,T)

array([1.87396991, 1.05820117, 1.23847727, 1.80522084])

In [62]:
def bubbleP(x, T):
    Pi= x*gamma(x,T)*p.Pvap(T)
    P=np.sum(Pi)
    return P, Pi/P

def bubbleT(x, P):

    def f(T):
        return bubbleP(x,T)[0]-P

    T=root_scalar(f, x0=350., method='secant').root
    Pi= x*gamma(x,T)*p.Pvap(T)
    P=np.sum(Pi)

    return T, Pi/P



In [76]:
P = 35347
R=8.314
# np.log(P/101325) = -p.Hvap(298)/R(1/T - 1/p.Tbn)

1/(1/p.Tbn-np.log(P/101325)*R/p.Hvap(298))

Array([301.68341783, 327.72429053, 332.50141705, 347.2907474 ], dtype=float64)

In [70]:
p.Tbn

array([329.4 , 351.44, 355.41, 373.15])

In [66]:
p.Hvap(373)

Array([27159.70592127, 36843.62636306, 37921.261929  , 40811.58769983],      dtype=float64)